In [12]:
%load_ext autoreload
%autoreload 2

In [1]:
from src.load_data import load_data
import jax
import jax.numpy as jnp
from src.svm import MultiClassKernelSVM
from src.kmeans import SphericalKMeans
import pandas as pd
from typing import Tuple
from src.kernels import RBF
import time
import pickle

In [2]:
data_folder = "data"
Xtr_flat, Ytr_flat, Xte_flat = load_data(data_folder, reshape=False)
print(f"Xtr {Xtr_flat.shape}; Ytr {Ytr_flat.shape}; Xte {Xte_flat.shape}")

Xtr, Ytr, Xte = load_data(data_folder, reshape=True)
print(f"Xtr {Xtr.shape}; Ytr {Ytr.shape}; Xte {Xte.shape}")

Xtr (5000, 3072); Ytr (5000,); Xte (2000, 3072)
Xtr (5000, 32, 32, 3); Ytr (5000,); Xte (2000, 32, 32, 3)


In [3]:
print(f"Xtr mean {Xtr_flat.mean(axis=0)} ; var {Xtr_flat.var(axis=0)}")

# should we z-scored the data ?
data_mean, data_std = Xtr_flat.mean(axis=0, keepdims=True), Xtr_flat.std(axis=0, keepdims=True)
Xtr_flat = (Xtr_flat - data_mean)/data_std
print(f"Xtr mean {Xtr_flat.mean(axis=0)} ; var {Xtr_flat.var(axis=0)}")

Xtr mean [-2.47982934e-04 -1.74261584e-03 -9.22621259e-05 ...  2.58250453e-03
  2.19476255e-03  3.04047441e-03] ; var [0.00167989 0.00118951 0.00118582 ... 0.00124412 0.00126839 0.00177964]
Xtr mean [-2.84217094e-18 -7.10542736e-18 -1.84741111e-17 ... -1.84741111e-17
 -8.52651283e-18 -1.24344979e-17] ; var [1. 1. 1. ... 1. 1. 1.]


### Kernel SVM

In [9]:
n_classes = 10
ntrain = 500

In [6]:
from src.kernels import RBF

In [14]:
sigma = jnp.sqrt(Xtr_flat.shape[1])
c = 1
kernel_func = RBF(sigma=sigma)
my_svm = MultiClassKernelSVM(num_classes=n_classes, kernel_func=kernel_func, c=c)
my_svm.fit(Xtr_flat[:ntrain], Ytr_flat[:ntrain])
pred = my_svm.predict(Xtr_flat[ntrain:])

print(f"Accuracy {jnp.mean(pred == Ytr_flat[ntrain:])}")

Function fit took 4.92 seconds
Function predict took 43.04 seconds
Accuracy 0.14733333885669708


In [16]:
ntrain = Xtr_flat.shape[0]
kernel_func = RBF(sigma=sigma)
my_svm = MultiClassKernelSVM(num_classes=n_classes, kernel_func=kernel_func, c=c)
my_svm.fit(Xtr_flat[:ntrain], Ytr_flat[:ntrain])

Function fit took 421.93 seconds


In [ ]:
Xte = (Xte_flat - data_mean)/data_std
Yte = my_svm.predict(Xte)
Yte = {"Prediction": Yte}

In [21]:
dataframe = pd.DataFrame(Yte)
dataframe.index += 1
dataframe.to_csv(f"{data_folder}/Yte.csv", index_label="Id")

### KFD

In [ ]:
from KFD import KFD, cosine_similarity

In [ ]:
def compute_accuracy(Y_true, Y_pred):
    return np.mean(Y_true == Y_pred)

In [ ]:
n_train = 4000
classifier = KFD(cosine_similarity)
classifier.train(Ytr[:n_train], Xtr[:n_train])

pred = classifier.predict(Xtr[:n_train])
accuracy = compute_accuracy(Ytr[:n_train], pred)
print("Accuracy on trained data:", accuracy * 100, "%")
pred = classifier.predict(Xtr[n_train:])
accuracy = compute_accuracy(Ytr[n_train:], pred)
print("Accuracy on unseen images:", accuracy * 100, "%")


In [ ]:
Yte = classifier.predict(Xte)
Yte = {"Prediction": Yte}
dataframe = pd.DataFrame(Yte)
dataframe.index += 1
dataframe.to_csv(data_folder / "Yte.csv", index_label="Id")
